In [1]:

import matplotlib.pyplot as plt
import pandas as pd
import json
from sqlalchemy import create_engine

Presets, agreed upon for all diagrams

In [2]:
x_incident = 3000
t_incident = 60

time_recording_after_incident = 560



presets for current run

In [3]:
# filter dates between 15th of March 2020 and 21th of March 2020
start_date = '2020-03-15'
end_date = '2020-03-21'

In [4]:
driver='postgresql'
username='dab_ds23241a_223'
dbname=username # it is the same as the username
password='Bh6KFgcWazMMCPvZ'
server='bronto.ewi.utwente.nl'
port='5432'

# Create an engine instance
alchemy_engine = create_engine(f'{driver}://{username}:{password}@{server}:{port}/{dbname}')

# Connect to PostgreSQL server
db_connection = alchemy_engine.connect()

# Read the data into a_df
a_df = pd.read_sql(f"select * from \"project\".\"Incidents_With_Cameras\"", db_connection)

# Check if 'before' and 'after' columns are already dictionaries
if a_df['before'].apply(type).eq(dict).all() and a_df['after'].apply(type).eq(dict).all():
    print('Columns are already dictionaries.')
else:
    # Convert the 'before' and 'after' columns from JSON strings back to dictionaries if they are not already
    a_df['before'] = a_df['before'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    a_df['after'] = a_df['after'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
# Close the database connection
db_connection.close()
print(a_df.shape)
a_df.head()


(266, 9)


,id,starttijd,type,gedetailleerd_type,primaire_locatie_lengtegraad,primaire_locatie_breedtegraad,segment,before,after
0,RWS02_0000157188_157188,2020-03-01 08:38:28,vehicle_obstruction,emergencyVehicle,4.366420,51.920952,north,{'RWS01_MONIBAS_0041hrl0706ra': 0.022954588858...,{'GEO0K_K_RWSTI357132': 0.16579609461072695}
1,RWS02_0000157197_157197,2020-03-01 10:00:57,vehicle_obstruction,emergencyVehicle,4.432840,51.928864,west-east,{'RWS01_MONIBAS_0201hrr0284ra': 0.094239550154...,{'RWS01_MONIBAS_0201hrr0285ra': 0.065785586736...
2,RWS02_0000157269_157269,2020-03-01 13:21:58,vehicle_obstruction,emergencyVehicle,4.456025,51.864422,east-west,{'RWS01_MONIBAS_0150vwy0557ra': 0.249505010285...,{'RWS01_MONIBAS_0150vwy0553ra': 0.229508095798...
3,RWS03_816372_1,2020-03-01 15:19:57,vehicle_obstruction,brokenDownVehicle,4.538568,51.909523,south,{'RWS01_MONIBAS_0160vwx0193ra': 0.896763594374...,{'RWS01_MONIBAS_0160vwx0202ra': 0.001663796286...
4,RWS02_0000157393_157393,2020-03-01 23:03:29,vehicle_obstruction,emergencyVehicle,4.439620,51.868599,east-west,{'RWS01_MONIBAS_0150vwy0544ra': 0.166980200322...,{'RWS01_MONIBAS_0150vwy0540ra': 0.210197772623...


This script creates the data which is used for the interpolation

In [5]:
# read camerasForText and extract the id's for camera's
cameraString=''
with open('camerasForTest.txt') as f:
    cameraString = f.readlines()[0]
    
    # remove /n
    cameraString = cameraString.replace('\n','')

cameras = []

if len(cameraString) > 0:
    cameras = cameraString.split(',')
else:
    print('No cameras found in file')
    
print(cameras)

# read the csv in chunks, and only include the rows where 




['RWS01_MONIBAS_0151hrl0533ra', ' RWS01_MONIBAS_0151hrl0538', 'RWS01_MONIBAS_0151hrl0543ra', 'RWS01_MONIBAS_0151hrl0548ra', 'RWS01_MONIBAS_0151hrl0554ra', 'RWS01_MONIBAS_0151hrl0560ra', 'RWS01_MONIBAS_0151hrl0567ra', 'RWS01_MONIBAS_0151hrl0572ra', 'RWS01_MONICA_00D00F08D446D005000D']


In [6]:
# create datapoints from trajectory for 5 hours

# transform camerasdata.xlsx into camerasdata.csv

# load camera bigdata, filter to only include cameras on the list, and between 5 hours. 







In [7]:
# # join with metadata dataset to get the location of the camera's
metadata = pd.read_csv('/Users/vree/Documents/UTwente/Masters UTwente/Data Science/DM Project/Speed-flow/Maart/intensiteit-snelheid-export 1 maart_metadata/intensiteit-snelheid-export_1 maart_metadata.csv')
# metadata.head(5)

# # keep id_meetlocatie, start_locatie_longitude, start_locatie_latitude
metadata = metadata[['id_meetlocatie', 'start_locatie_longitude', 'start_locatie_latitude']]

# # remove metadata if it does not contain the camera id
metadata = metadata[metadata['id_meetlocatie'].isin(cameras)]

# save metadata to csv
metadata.to_csv('metadata.csv', index=False)


# # show columns 
# metadata.columns

# read metadata
metadata = pd.read_csv('metadata.csv')

metadata = metadata[metadata['id_meetlocatie'].isin(cameras)]

metadata.head(5)
metadata.drop_duplicates(subset='id_meetlocatie', keep='first', inplace=True)

print(metadata.shape)

/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_32230/4177329327.py:2: DtypeWarning: Columns (26,39,40,42,49,50,52,53,55) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('/Users/vree/Documents/UTwente/Masters UTwente/Data Science/DM Project/Speed-flow/Maart/intensiteit-snelheid-export 1 maart_metadata/intensiteit-snelheid-export_1 maart_metadata.csv')


(8, 3)


In [8]:
# read camerasData.xlsx
df = pd.read_csv('cameradata2.csv')

df.shape


(32372, 15)

In [9]:
# Create a copy of the DataFrame with only the required columns to avoid SettingWithCopyWarning
df = df[['id_meetlocatie', 'start_meetperiode', 'eind_meetperiode', 'gem_snelheid']].copy()

# Replace the negative values with 0 using .loc to ensure the operation is done in place
df.loc[df['gem_snelheid'] < 0, 'gem_snelheid'] = 0

# Now count the number of rows where 'gem_snelheid' is lower than 0 (should be 0 after the replacement)
negative_speed_count = df[df['gem_snelheid'] < 0].shape[0]
print(negative_speed_count)
df.head(5)

# aggregate instances where id_meetlocatie, start_meetperiode, eind_meetperiode are the same, and take the mean of the gem_snelheid
df = df.groupby(['id_meetlocatie', 'start_meetperiode', 'eind_meetperiode']).agg({'gem_snelheid': 'mean'}).reset_index()
df.head(5)

df.shape



0


(9593, 4)

In [10]:



# join the metadata with the data frame on id_meetlocatie and remove one of the id_meetlocatie columns
df = df.merge(metadata, on='id_meetlocatie', how='left')
df.head(5)
df.shape




(9593, 6)

In [11]:
df.head()

,id_meetlocatie,start_meetperiode,eind_meetperiode,gem_snelheid,start_locatie_longitude,start_locatie_latitude
0,RWS01_MONIBAS_0151hrl0533ra,2020-03-04 02:00:00,2020-03-04 02:01:00,30.666667,4.42663,51.87127
1,RWS01_MONIBAS_0151hrl0533ra,2020-03-04 02:01:00,2020-03-04 02:02:00,0.000000,4.42663,51.87127
2,RWS01_MONIBAS_0151hrl0533ra,2020-03-04 02:02:00,2020-03-04 02:03:00,28.000000,4.42663,51.87127
3,RWS01_MONIBAS_0151hrl0533ra,2020-03-04 02:03:00,2020-03-04 02:04:00,0.000000,4.42663,51.87127
4,RWS01_MONIBAS_0151hrl0533ra,2020-03-04 02:04:00,2020-03-04 02:05:00,0.000000,4.42663,51.87127


In [12]:
import numpy as np

In [13]:
def haversine(lon1, lat1, lon2, lat2):
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371 # Radius of Earth in kilometers. Use 3956 for miles
    return c * r

In [14]:
# calculate distance between camera's . the first camera is the start location, last camera is the end location

# camera's dataframe
cameras_df = df.drop_duplicates(subset='id_meetlocatie')



# order the camera's based on the order of cameras
cameras_df['order'] = cameras_df['id_meetlocatie'].apply(lambda x: cameras.index(x))

# sort the dataframe based on the order
cameras_df = cameras_df.sort_values(by='order').reset_index(drop=True)

# Assuming you have a DataFrame named cameras_df and a haversine function defined

print(cameras_df.shape)
# Initialize the distance to the first camera column with zeros
cameras_df['distance_to_first_camera'] = 0.0

# Calculate the distance between each camera and the previous camera, and accumulate these distances
# to calculate the distance to the first camera
accumulated_distance = 0.0


for i in range(1, len(cameras_df)):
    distance_to_previous_camera = haversine(cameras_df.iloc[i-1, 4], cameras_df.iloc[i-1, 5], 
                                            cameras_df.iloc[i, 4], cameras_df.iloc[i, 5])
    accumulated_distance += distance_to_previous_camera
    cameras_df.loc[i, 'distance_to_first_camera'] = accumulated_distance

cameras_df.head()




    





(8, 7)


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_32230/2854836967.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cameras_df['order'] = cameras_df['id_meetlocatie'].apply(lambda x: cameras.index(x))


,id_meetlocatie,start_meetperiode,eind_meetperiode,gem_snelheid,start_locatie_longitude,start_locatie_latitude,order,distance_to_first_camera
0,RWS01_MONIBAS_0151hrl0533ra,2020-03-04 02:00:00,2020-03-04 02:01:00,30.666667,4.42663,51.87127,0,0.000000
1,RWS01_MONIBAS_0151hrl0543ra,2020-03-04 02:00:00,2020-03-04 02:01:00,0.000000,4.44046,51.86817,2,1.010165
2,RWS01_MONIBAS_0151hrl0548ra,2020-03-04 02:00:00,2020-03-04 02:01:00,0.000000,4.44710,51.86655,3,1.500372
3,RWS01_MONIBAS_0151hrl0554ra,2020-03-04 02:00:00,2020-03-04 02:01:00,28.000000,4.45405,51.86468,4,2.020919
4,RWS01_MONIBAS_0151hrl0560ra,2020-03-04 02:00:00,2020-03-04 02:01:00,28.666667,4.46276,51.86270,5,2.658242


In [15]:
cameras_df = cameras_df[['id_meetlocatie','distance_to_first_camera']]


# inner join df with cameras_df on id_meetlocatie
df = df.merge(cameras_df, on='id_meetlocatie', how='left')

# add 2km to the distance_to_first_camera to get the distance to the start of the road
df['distance_to_start'] = df['distance_to_first_camera'] + 1


df.shape

df.head(5)




,id_meetlocatie,start_meetperiode,eind_meetperiode,gem_snelheid,start_locatie_longitude,start_locatie_latitude,distance_to_first_camera,distance_to_start
0,RWS01_MONIBAS_0151hrl0533ra,2020-03-04 02:00:00,2020-03-04 02:01:00,30.666667,4.42663,51.87127,0.0,1.0
1,RWS01_MONIBAS_0151hrl0533ra,2020-03-04 02:01:00,2020-03-04 02:02:00,0.000000,4.42663,51.87127,0.0,1.0
2,RWS01_MONIBAS_0151hrl0533ra,2020-03-04 02:02:00,2020-03-04 02:03:00,28.000000,4.42663,51.87127,0.0,1.0
3,RWS01_MONIBAS_0151hrl0533ra,2020-03-04 02:03:00,2020-03-04 02:04:00,0.000000,4.42663,51.87127,0.0,1.0
4,RWS01_MONIBAS_0151hrl0533ra,2020-03-04 02:04:00,2020-03-04 02:05:00,0.000000,4.42663,51.87127,0.0,1.0


In [16]:
# print unique values for the distance_to_first_camera
print(df['distance_to_first_camera'].unique())
print(df['start_meetperiode'].unique())
print()

[0.         1.01016543 1.50037206 2.02091902 2.65824235 3.4070625
 3.89726542 4.6084661 ]
['2020-03-04 02:00:00' '2020-03-04 02:01:00' '2020-03-04 02:02:00' ...
 '2020-03-04 21:57:00' '2020-03-04 21:58:00' '2020-03-04 21:59:00']



In [17]:
#get for every camera the earliest time recording
df['start_meetperiode'] = pd.to_datetime(df['start_meetperiode'])

minimum_values_per_camera= df.groupby(['id_meetlocatie']).agg({'start_meetperiode': 'min'}).reset_index()

minimum_value = minimum_values_per_camera['start_meetperiode'].min()

print(minimum_value)

minimum_values_per_camera.head()

# calculate the minutes from the minimum time 
df['minutes_from_min'] = (df['start_meetperiode'] - minimum_value).dt.total_seconds() / 60
df['distance_to_start'] = df['distance_to_start'] * 1000

datapoints = df[['minutes_from_min', 'distance_to_start', 'gem_snelheid']]

# swap order
datapoints = datapoints[['distance_to_start', 'minutes_from_min', 'gem_snelheid']]

# rename columns to x, t, v
datapoints.columns = ['x', 't', 'v']

# remove cases where the entry is not a number
datapoints = datapoints[~datapoints['v'].isna()]
datapoints = datapoints[~datapoints['x'].isna()]
datapoints = datapoints[~datapoints['t'].isna()]

# write to csv file
datapoints.to_csv('datapointsForInterpolation2.csv', index=False)

datapoints.head()

2020-03-04 02:00:00


,x,t,v
0,1000.0,0.0,30.666667
1,1000.0,1.0,0.000000
2,1000.0,2.0,28.000000
3,1000.0,3.0,0.000000
4,1000.0,4.0,0.000000
